In [ ]:
%%capture
!pip install chromadb
!pip install -U bitsandbytes
!pip install llama-cpp-python
!pip install rank_bm25 nltk

# Objectives

Building a simple RAG for question answering based on a light quantized LLama3.2 1B model. The goal is to answer accurately questions concerning Warhammer 40K rules.

## Overview

First we will import Llama 3.2 and try some templatting and chat with the model.

Secondly, we will experiment with [ChromaDB](https://docs.trychroma.com/getting-started) and build a first RAG.

Finally, we will be using the headers and BM25 to try and improve the retriever.

# Imports

In [ ]:
import chromadb
import json
import uuid

from llama_cpp import Llama
from transformers import AutoModelForCausalLM, AutoTokenizer
from jinja2 import Template


from rank_bm25 import BM25Okapi
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
nltk.download('stopwords')

import spacy

# Large Language Model

## CPU implementation

In [ ]:
llm = Llama.from_pretrained(
      repo_id="bartowski/Llama-3.2-1B-Instruct-GGUF",
      filename="*Q8_0.gguf",
      verbose=False,
      n_ctx=25000,
)

In [ ]:
def llama_cpp_generate(prompt):
  output = llm(prompt, max_tokens=300)
  return output["choices"][0]["text"]

llama_cpp_generate("What is the capital of France ?")

## GPU Implementation

In [ ]:
# Import model
tokenizer = AutoTokenizer.from_pretrained("unsloth/Llama-3.2-1B-Instruct")
model = AutoModelForCausalLM.from_pretrained("unsloth/Llama-3.2-1B-Instruct")

In [ ]:
def llm_complete(prompt, max_tokens=300):
    inputs = tokenizer(prompt, return_tensors="pt")
    max_length = len(inputs["input_ids"][0]) + max_tokens
    outputs = model.generate(**inputs, max_length=max_length)
    generated_tokens = outputs[0][len(inputs["input_ids"][0]):]  # Exclude the input prompt tokens
    answer = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    return answer

## Initial LLM experiments

### Prompt Template

In [ ]:
prompt_template = Template(
    """
    <|begin_of_text|><|start_header_id|>system<|end_header_id|>
    {{role}}<|eot_id|><|start_header_id|>user<|end_header_id|>
    {{input}}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """
)

### Playing with the LLM

In [ ]:
input = {'role':'you are a depressed miserable sad clown' ,'input': 'tell me a joke !'}
input_2 = {'role':'you are a clown for children' ,'input': 'tell me a joke !'}

LLama cpp

In [ ]:
prompt = prompt_template.render(role=input["role"], input=input["input"])
llama_cpp_generate(prompt)

In [ ]:
prompt_2 = prompt_template.render(role=input_2["role"], input=input_2["input"])
llama_cpp_generate(prompt_2)

Unsloth Llama 3.2

In [ ]:
llm_complete(prompt)

In [ ]:
llm_complete(prompt_2)

What do you think ?




It behaves quite differently so the role is important

# Retriever

In [ ]:
import chromadb.utils.embedding_functions as embedding_functions

In [ ]:
%%capture
client = chromadb.Client()
stf_function = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

In [ ]:
collection = client.create_collection(name="warhammer_40k",
                                      metadata={"hnsw:space": "cosine"},
                                      embedding_function=stf_function)

In [ ]:
!unzip /content/my_first_rag_data.zip

In [ ]:
with open('chunks.json', 'r') as f:
    chunks = json.load(f)

In [ ]:
# Adding the documents to the collection
collection.add(
    documents=[doc["page_content"] for doc in chunks],
    ids=[str(i) for i in range(len(chunks))],
    metadatas=[doc["metadata"] for doc in chunks]
)

In [ ]:
question = "What is a visible unit ?"

# Performing a query
collection.query(
    query_texts=[question],
    n_results=4
)

Creating a retrieval function wrapping the ChromaDB query and returning an adapted format.

In [ ]:
# returns list[dict[str:str]], each dictionnary represents a chunk
def retrieve(question, n_results=5):
    res_query = collection.query(
        query_texts=[question],
        n_results=n_results
    )

    chunk_list = [{"header":res_query["metadatas"][0][i],
                   "text":res_query["documents"][0][i]} for i in range(n_results)]

    return chunk_list

## Rag template

Create a RAG template in Jinja

In [ ]:
# do a loop inside
rag_template = Template(
  """
  <|begin_of_text|>

  <|start_header_id|>system<|end_header_id|>
  {{role}}
  <|eot_id|>

  <|start_header_id|>user<|end_header_id|>

  Use the context and only the context to answer the following question:

  Question:

  {{question}}

  Context:
  {% for chunk in chunks %}
  {% for key, value in chunk.header.items() %}{{ key }}: {{ value  }} {% endfor %}
  text: {{chunk.text}}
  {% endfor %}
  <|eot_id|>

  <|start_header_id|>assistant<|end_header_id|>
  """
)

In [ ]:
print(rag_template.render(**{
    'role': 'you are an experienced wargame player',
    'question': "What is a visible unit ?",
    'chunks':[{'header': {'header1':'toto'},'text':'ctx1'},{'header': {'header1':'tato', 'header2':'tato'},'text':'ctx2'},{'header': {'header1':'tato'}, 'text':'ctx3'}]
}))

Create a template that will create the prompt using the question and chunks.

In [ ]:
# returns a prompt
def prompt_generation(question, chunks):
    return rag_template.render(**{
      'role': 'you are an experienced wargame player',
      'question': question,
      'chunks': chunks
    })

In [ ]:
print(prompt_generation(question, retrieve(question)))

# Full Rag

Create functions to perform the full RAG pipeline, you may create a function for the CPU and another one for the GPU.

In [ ]:
question_0 = "What is a visible unit ?"
question_1 = 'What are the limitations associated to the advance mouvement rule ?'
question_2 = 'Is there a stratagem that can be used to reroll a failed dice role?'
question_3 = 'Explain the Comand Re-roll stratagem'

In [ ]:
def full_rag_cpu(question, n_results=4):
  prompt = prompt_generation(question,retrieve(question))
  answer = llama_cpp_generate(prompt)
  return answer

In [ ]:
full_rag_cpu(question_0)

In [ ]:
full_rag_cpu(question_1)

In [ ]:
full_rag_cpu(question_2)

In [ ]:
full_rag_cpu(question_3)

<hr>

In [ ]:
def full_rag_gpu(question, n_results=4):
    prompt = prompt_generation(question,retrieve(question))
    answer = llm_complete(prompt)
    return answer

In [ ]:
full_rag_gpu(question_0)

In [ ]:
full_rag_gpu(question_1)

In [ ]:
full_rag_gpu(question_2)

In [ ]:
full_rag_gpu(question_3)

# Adding a reranker
The results Might not be satisfactory for some questions.

In order to better use the header extraction, we will rerank the chunks using BM25 over the headers.

#### Tokenization

In [ ]:
eng_stopwords_set = set(stopwords.words('english'))
nlp = spacy.load("en_core_web_sm")

In [ ]:
def tokenize_without_stopwords(s, stopwords_set=set()):
  return [token.text for token in nlp(s) if token.text not in stopwords_set]

# Testing tokenization
test_tokenized = tokenize_without_stopwords(question_2, eng_stopwords_set)
test_tokenized

#### Stemming

In [ ]:
stemmer = SnowballStemmer(language="english")

# returns list[str]
def stem(l_tokens):
  return [stemmer.stem(token) for token in l_tokens]

test_stemmed = stem(test_tokenized)
test_stemmed

#### Preprocessing

In [ ]:
def metadata_preprocessing(metadata):
    # medata: list[str]
    tokenized = [tokenize_without_stopwords(s) for s in metadata]
    preprocessed_headers = [stem(l_tokens) for l_tokens in tokenized]
    return preprocessed_headers # list[list[str]]

def query_preprocessing(query):
    # query: str
    tokenized = tokenize_without_stopwords(query)
    preprocessed_queries = stem(tokenized)
    return preprocessed_queries # list[str]

#### BM25 engine

In [ ]:
def rerank_chunks(question, chunks, n_results=5):
    headers = ['#'.join(chunk["header"].values()) for chunk in chunks] # list[str]
    preprocessed_headers = metadata_preprocessing(headers)
    preprocessed_query = query_preprocessing(question)

    bm25_model = BM25Okapi(preprocessed_headers)

    scores = bm25_model.get_scores(preprocessed_query)
    chunks_score_list = list(zip(chunks, scores)) # list[ tuple( dict[str:str], float ) ]
    chunks_score_list.sort(key=lambda x: x[1], reverse=True)

    best_chunks = chunks_score_list[:n_results]

    return best_chunks # list[ tuple( dict[str:str], float ) ], returns best chunks and their score

#### Tests

In [ ]:
retrieved_chunks = retrieve(question_0, n_results=100) # list[dict[str:str]]
reranked_chunks = rerank_chunks(question_0, retrieved_chunks, n_results=10) # list[ tuple( dict[str:str],float ) ]
for rr_chk in reranked_chunks:
    print('score: {}'.format(rr_chk[1]))
    print('header: {}'.format('#'.join(rr_chk[0]['header'].values())))
    print('text: {}'.format(rr_chk[0]['text']))
    print('-'*100)

In [ ]:
retrieved_chunks = retrieve(question_1, n_results=100) # list[dict[str:str]]
reranked_chunks = rerank_chunks(question_1, retrieved_chunks, n_results=10) # list[ tuple( dict[str:str],float ) ]
for rr_chk in reranked_chunks:
    print('score: {}'.format(rr_chk[1]))
    print('header: {}'.format('#'.join(rr_chk[0]['header'].values())))
    print('text: {}'.format(rr_chk[0]['text']))
    print('-'*100)

In [ ]:
retrieved_chunks = retrieve(question_2, n_results=100)
reranked_chunks = rerank_chunks(question_2, retrieved_chunks, n_results=10)
for rr_chk in reranked_chunks:
    print('score: {}'.format(rr_chk[1]))
    print('header: {}'.format('#'.join(rr_chk[0]['header'].values())))
    print('text: {}'.format(rr_chk[0]['text']))
    print('-'*100)

In [ ]:
question_3 = 'Explain the Comand Re-roll stratagem'
retrieved_chunks = retrieve(question_3, n_results=100)
reranked_chunks = rerank_chunks(question_3, retrieved_chunks, n_results=10)
for rr_chk in reranked_chunks:
    print('score: {}'.format(rr_chk[1]))
    print('header: {}'.format('#'.join(rr_chk[0]['header'].values())))
    print('text: {}'.format(rr_chk[0]['text']))
    print('-'*100)

## RAG with reranker

In [ ]:
history = [] # list[dict[str:str]]

def full_rag_reranker_cpu(question, n_results=5):
  retrieved_chunks = retrieve(question, n_results=100) # list[dict[str:str]]
  reranked_chunks_with_scores = rerank_chunks(question, retrieved_chunks, n_results=5) # list[ tuple( dict[str:str],float ) ]
  reranked_chunks = [tuple_chk_score[0] for tuple_chk_score in reranked_chunks_with_scores]
  prompt = prompt_generation(question,reranked_chunks)
  answer = llama_cpp_generate(prompt)
  return answer

In [ ]:
question = "How can I win the game?"

Testing with default 5 chunks as context

In [ ]:
full_rag_cpu(question)

In [ ]:
full_rag_reranker_cpu(question)

<hr>

In [ ]:
def full_rag_reranker_gpu(question, n_results=5):
  retrieved_chunks = retrieve(question, n_results=100) # list[dict[str:str]]
  reranked_chunks_with_scores = rerank_chunks(question, retrieved_chunks, n_results=5) # list[ tuple( dict[str:str],float ) ]
  reranked_chunks = [tuple_chk_score[0] for tuple_chk_score in reranked_chunks_with_scores]
  prompt = prompt_generation(question,reranked_chunks)
  answer = llm_complete(prompt)
  return answer

In [ ]:
full_rag_gpu(question)

In [ ]:
full_rag_reranker_gpu(question)

The answer is better using full_rag_reranker: it is more detailed and relevant

# Chatting with the rulebook

At this point, we created a single question/answer turn RAG. It can be usefull for some applications to allow conversations with documents.

In [ ]:
history = [] # list[dict[str:str]], contains the chat history

In [ ]:
# do a loop inside
chat_rag_template = Template(
  """
  <|begin_of_text|>

  <|start_header_id|>system<|end_header_id|>
  {{role}}
  <|eot_id|>

  {% for chat in history %}
  <|start_header_id|>user<|end_header_id|>
  {{chat.h_question}}
  <|eot_id|>

  <|start_header_id|>assistant<|end_header_id|>
  {{chat.h_answer}}
  <|eot_id|>
  {% endfor %}

  <|start_header_id|>user<|end_header_id|>

  Use the context and only the context to answer the following question:

  Question:

  {{question}}

  Context:
  {% for chunk in chunks %}
  {% for key, value in chunk.header.items() %}{{ key }}: {{ value  }} {% endfor %}
  text: {{chunk.text}}
  {% endfor %}
  <|eot_id|>

  <|start_header_id|>assistant<|end_header_id|>
  """
)

In [ ]:
def chat_prompt_generation(question, chunks, history):
    return chat_rag_template.render(**{
    'role': 'you are an experienced wargame player',
    'question': question, # str
    'chunks': chunks, # list[dict[str:str]]
    'history': history # list[dict[str:str]]
    })

In [ ]:
def chat_with_rag_gpu(question, n_results=5, clear_history=False):
  if clear_history: history.clear()

  retrieved_chunks = retrieve(question, n_results=100) # list[dict[str:str]]
  reranked_chunks_with_scores = rerank_chunks(question, retrieved_chunks, n_results=5) # list[ tuple( dict[str:str],float ) ]
  reranked_chunks = [tuple_chk_score[0] for tuple_chk_score in reranked_chunks_with_scores]
  prompt = chat_prompt_generation(question, reranked_chunks, history)
  answer = llm_complete(prompt)

  # Only add question and answer to limit going over token limit in prompt
  current_chat = {"h_question":question, "h_answer":answer}
  history.append(current_chat)

  return answer

In [ ]:
chat_with_rag_gpu(question, clear_history=True)

In [ ]:
chat_with_rag_gpu("What are the victory conditions ?")

In [ ]:
chat_with_rag_gpu("What objective markers are there ?")